# An Explain Report

This is a POC for an explain report: given a number at time T1 and time T2, introspect the clocks involved to produce a simple linear breakdown
of what changed on various timelines to account for the change in the number between T1 and T2.

The report is an ordered sequence of clock changes: the sum of changes should equal to the total change. The order of application is important: change the
ordering, and the resultant values may change. It's not a Jacobian-type report with T1 deltas and no ordering.

The real reason for this workbook is to start working on the node class optimizations and metadata management.

## Footnotes

Footnotes are one of the simplest types of metadata:
* Any computation (even a simple read of a constant value) may declare it has footnotes
* Every computation gets the footnote of all its inputs

Footnoting is based on a number of observations about large software systems (say, 1M LOC or more):
* Any complex report (i.e. computation output) will generally be somewhat wrong, misleading, or out of date.
* If you want 100% correctness and truth, your report will raise an exception every time you run it.
* If you just want it to run, you'll get a result, but some log file on some machine in some compute farm will have a message explaining why your result is wrong.
* And you won't see that message.
* And that message may have leaked information that bad actors can read.
* And you won't trust this system, so your own developers will copy the underlying data and write their own version of the report with all the same issues.
* Now you don't have a problem, but your company now has two problems.

Footnotes summary:
* Are the documentation of problems from the producer's point of view
* Make no claim about usability of results, etc, for a specific consumer
* May be programmatically removed/condensed/replaced at controlled code points


In [1]:
from mand.core import Entity, node, Context
from mand.core import ObjectDb, Timestamp, getNode, _tr
from mand.core import ProfileMonitor
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import find
from mand.demos.trading import makeWorld, bookSomeTrades

db = ObjectDb()

from mand.lib.dbsetup import setUpDb
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x10797ed90>: 241, mem=True, ro=False: entities=9, map=2


In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


# Book some trades

This time, we throw a few thousand in...

In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

# An Explain Report

This is a mix of abstractions at the Core, DBA, and BA, and User levels. But, it does the job for now...

In [4]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        # XXX - modifying tweaks in place is a bit evil
        # This is only safe because I know Context() effectively copies, so this works
        # for now.
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            txt = '\n'.join( [ '1. %s' % f for f in footnotes])
            displayMarkdown(txt)
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 5236.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [5]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

# Footnotes

Note the report calculation has run, but attached appropriate caveats to the output:

In [6]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times: /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Report:data|1|18,655,118|14|18,655,118|GetValue
|Report:data|1|18,655,104|467|18,655,104|GetValue/Calc
|TradingContainer:NPV|11|18,590,540|115|1,690,049|GetValue
|TradingContainer:NPV|11|18,590,424|1,857|1,690,038|GetValue/Calc
|Portfolio:items|121|18,419,767|1,010|152,229|GetValue
|Portfolio:items|121|18,419,690|20,151|152,228|GetValue/Calc
|Workbook:items|1,104|17,402,680|8,963|15,763|GetValue
|Workbook:items|1,100|17,393,717|8,120,139|15,812|GetValue/Calc
|Root:Clocks|2|10,914,302|60,257|5,457,151|Context
|TradeOpenEvent|1,014|4,534,263|4,534,263|4,471|Db.Get
|TradeOpenEvent:ticket|22,286|4,126,852|93,194|185|GetValue
|TradingTicket|1,014|4,033,658|4,033,658|3,977|Db.Get
|Root:Activity Portfolio|1|1,133,769|40|1,133,769|Context
|Root:Amend MarketData|1|1,079,775|40|1,079,775|Context
|Portfolio:children|242|991,023|1,510|4,095|GetValue
|Portfolio:children|121|989,513|489,945|8,177|GetValue/Calc
|Root:End|1|808,483|38|808,483|Context
|Root:Start|1|803,009|20|803,009|Context
|Root:Activity Trading|1|787,055|45|787,055|Context
|Root:Amend Portfolio|1|783,949|37|783,949|Context
|Root:Start breaks|1|783,776|37|783,776|Context
|Root:Amend Trading|1|781,308|43|781,308|Context
|Root:Activity MarketData|1|775,712|41|775,712|Context
|PortfolioUpdateEvent:children|121|417,450|2,457|3,450|GetValue
|TradingBook|102|384,067|384,067|3,765|Db.Get
|Equity:NPV|15|168,613|146|11,240|GetValue
|Equity:NPV|15|168,467|372|11,231|GetValue/Calc
|MarketInterface:spot|15|158,855|118|10,590|GetValue
|MarketInterface:spot|15|158,736|377|10,582|GetValue/Calc
|ExternalRefData:state|15|150,708|106|10,047|GetValue
|ExternalRefData:state|15|150,602|250|10,040|GetValue/Calc
|RefData:state|15|150,352|114|10,023|GetValue
|RefData:state|15|150,238|63,977|10,015|GetValue/Calc
|Clock:cutoffs|2,480|102,696|7,012|41|GetValue
|Clock:cutoffs|20|95,742|432|4,787|GetValue/Calc
|Clock:parent|20|95,218|133|4,760|GetValue
|Clock:parent|20|95,084|78,669|4,754|GetValue/Calc
|_WorkItemEvent:book2|22,286|88,494|80,610|3|GetValue
|_WorkItemEvent:book1|22,286|84,500|84,500|3|GetValue
|TradeOpenEvent:quantity|22,286|74,809|74,809|3|GetValue
|_WorkItemEvent:item|11,143|67,434|59,829|6|GetValue
|TradeOpenEvent:premium|11,143|59,942|55,364|5|GetValue
|TradeOpenEvent:action|11,143|58,427|58,427|5|GetValue
|PortfolioUpdateEvent|12|58,239|58,239|4,853|Db.Get
|Event:amends|11,340|48,326|48,326|4|GetValue
|TradeOpenEvent:unitPrice|11,143|48,140|48,140|4|GetValue
|RefDataUpdateEvent|9|42,770|42,770|4,752|Db.Get
|TradingPortfolio|10|38,809|38,809|3,880|Db.Get
|TradingBook:clock|2,200|35,401|17,012|16|GetValue
|Clock|5|19,570|19,570|3,914|Db.Get
|TradingBook:clock|1,100|18,389|14,590|16|GetValue/Calc
|Equity:refdata|15|9,238|129|615|GetValue
|Equity:refdata|15|9,109|509|607|GetValue/Calc
|MarketInterface|2|8,512|8,512|4,256|Db.Get
|ClockEvent:parent|8|8,234|68|1,029|GetValue
|Portfolio:clock|242|7,753|1,946|32|GetValue
|MarketInterface:source|15|7,650|96|510|GetValue
|Equity|2|7,604|7,604|3,802|Db.Get
|MarketInterface:source|15|7,554|320|503|GetValue/Calc
|ClockEvent|2|7,551|7,551|3,775|Db.Get
|MarketDataSource|2|7,077|7,077|3,538|Db.Get
|Portfolio:clock|121|5,807|1,522|47|GetValue/Calc
|Portfolio:books|231|5,233|1,403|22|GetValue
|ForwardCashflow|1|4,577|4,577|4,577|Db.Get
|Portfolio:books|121|4,106|3,498|33|GetValue/Calc
|MarketDataSource:clock|30|3,731|226|124|GetValue
|MarketDataSource:clock|15|3,504|184|233|GetValue/Calc
|RootClock|1|3,479|3,479|3,479|Db.Get
|Entity:clock|40|474|294|11|GetValue
|RefDataUpdateEvent:data|58|220|220|3|GetValue
|ForwardCashflow:NPV|11|186|111|16|GetValue
|Entity:clock|20|180|180|9|GetValue/Calc
|MarketInterface:sourceName|15|155|114|10|GetValue
|RootClock:cutoffs|52|139|139|2|GetValue
|Equity:assetName|15|87|87|5|GetValue
|ForwardCashflow:NPV|11|74|74|6|GetValue/Calc
|MarketInterface:sourceName|15|41|41|2|GetValue/Calc
|Report:valuable|1|5|5|5|GetValue
|Report:ts2|1|3|3|3|GetValue
|Report:ts1|1|3|3|3|GetValue